In [1]:
import os,sys, math, ROOT
from array import array
import numpy as np
# %jsroot on

#Importing a helper file where some functions are defined
from helper import *

#Input file
inputfile = ROOT.TFile("../Data/data.root","READ")

#Output file
outputfile = ROOT.TFile("output.root","RECREATE")

#Accessing the "TTree" from root that contains the data (one entry per event)
tree = inputfile.Get("ntuplizer/tree")

nentries = tree.GetEntries()
print("Number of entries: ", nentries)

Welcome to JupyROOT 6.26/06
Number of entries:  104317


In [2]:
#Example of histograms 
#The arguments are the following: 
#"name", "title;x axis title;yaxis title", nb of bins, min value, max value

# hist_npv = ROOT.TH1F("hist_NPV", "NPV;Number of reconstructed vertices;Events", 100, 0, 100)
# hist_n_e = ROOT.TH1F("hist_n_e",  ";Number of electrons detected;Events", 10, 0, 10)
# hist_n_mu = ROOT.TH1F("hist_n_mu",";Number of muons dectected;Events", 10, 0, 10)

hist_massfourleptons = ROOT.TH1F("hist_massfourleptons",";Four Lepton Invariant Mass;Events", 5000, 0, 500)

In [3]:
#Loop over entries
for i in range(0,nentries):
    tree.GetEntry(i) # Load entry number i
    n_e,n_mu_minus,n_mu_p,n_p = 0,0,0,0   

    for lep in range(0,len(tree._lPt)): # Loop over leptons per event
        if tree._lpdgId[lep] == 11 and n_e != 1: # Detect if lepton == electron
            n_e += 1
            Index_e = lep
        elif tree._lpdgId[lep] == -11 and n_p != 1: # Detect if lepton == positron
            n_p += 1
            Index_p = lep
        elif tree._lpdgId[lep] == 13 and n_mu_minus != 1: # Detect if lepton == muon-
            n_mu_minus += 1
            Index_mu_minus = lep
        elif tree._lpdgId[lep] == -13 and n_mu_p != 1: # Detect if lepton == muon+
            n_mu_p +=1
            Index_mu_plus = lep
        if (n_e,n_p,n_mu_minus,n_mu_p) == (1,1,1,1): # If all sought-after leptons are detected, break out of the loop
            invariant_mass = invariantmass4l(tree,Index_e,Index_p,Index_mu_minus,Index_mu_plus)
            hist_massfourleptons.Fill(invariant_mass)
    

Let us notice straight away there are close 47% more muons detected than electrons detected. *Does this mean that dectecting muons is statistically more interesting that counting electrons?*

In [4]:
#Draw the histogram and the canvas
c = ROOT.TCanvas("canvas","c_NPV",600,600)
# hist_n_mu.SetLineColor(ROOT.kRed)
# hist_n_e.Draw()
# hist_n_mu.Draw("same")
# hist_npv.Draw()
# c.Draw()
# savehisto(outputfile, hist_n_mu, "hist_n_mu_log")
# savehisto(outputfile,hist_n_e,"hist_n_e_log")

savehisto(outputfile,hist_massfourleptons,"hist_2e2mu_InvariantMass",logy=True)

Info in <TCanvas::Print>: pdf file pdfs/hist_2e2mu_InvariantMass.pdf has been created


For the duration of the internship, we shall only consider events involving 2 electrons and 2 muons. Hence, let us apply a filter and select only events for which there are at lea